<a href="https://colab.research.google.com/github/Prakadeeswaran05/ICP/blob/main/ICP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install open3d

In [2]:
import numpy as np

# first point cloud p
n = 1000
x = np.random.random(n) * 2 - 1
y = np.random.random(n) * 2 - 1
z = 4*( x * x + y * y)
p = np.c_[x, y, z]

def rotate_3d(p, yaw, pitch, roll):
    a = np.array([[+np.cos(yaw), -np.sin(yaw), 0],
                  [+np.sin(yaw), +np.cos(yaw), 0],
                  [0, 0, 1]])
    b = np.array([[+np.cos(pitch), 0, +np.sin(pitch)],
                  [0, 1, 0],
                  [-np.sin(pitch), 0, +np.cos(pitch)]])
    c = np.array([[1, 0, 0],
                  [0, +np.cos(roll), -np.sin(roll)],
                  [0, +np.sin(roll), +np.cos(roll)]])
    return a.dot(b.dot(c.dot(p.T))).T


# second point cloud q
q = rotate_3d(p, np.pi/4, np.pi/4, np.pi/4)
q += np.array([1.25, 1.50, 1.75])

In [9]:
import open3d as o3d
import plotly.graph_objects as go
# Open3d point cloud objects

color1=np.tile(np.array([[1, 0.5, 0]]), (len(p), 1))
po = o3d.geometry.PointCloud()
po.points = o3d.utility.Vector3dVector(p)
po.colors =o3d.utility.Vector3dVector(color1)
points1=np.asarray(po.points)
color1= np.asarray(po.colors)

color2=np.tile(np.array([[0.5, 0, 1]]), (len(q), 1))
qo = o3d.geometry.PointCloud()
qo.points = o3d.utility.Vector3dVector(q)
qo.colors =o3d.utility.Vector3dVector(color2)
points2 = np.asarray(qo.points)
color2=np.asarray(qo.colors)

points=np.concatenate((points1,points2))
colors=np.concatenate((color1,color2))
fig = go.Figure(
  data=[
    go.Scatter3d(
      x=points[:,0], y=points[:,1], z=points[:,2],
      mode='markers',
      marker=dict(size=1,color=colors)
)
],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False)
)
)
)
fig.show()


In [4]:
def cov(p, q, matches=None):
    n = p.shape[1]
    assert q.shape[1] == n, "dimension mismatch in cross_cov()"

    if matches is None:
        matches = [(i, i) for i in range(p.shape[0])]

    cov = np.zeros((n, n))
    matches = np.array(matches)
    pi = p[matches[:, 0]]
    qj = q[matches[:, 1]]
    cov = np.sum(pi[:, :, np.newaxis] * qj[:, np.newaxis, :], axis=0)

    return cov



In [5]:
from scipy.spatial import cKDTree as kdtree
def transform(cc,pm,q,qm):
  u, s, v = np.linalg.svd(cc)
  r = u.dot(v)
  t = pm - r.dot(qm)
  q1 = r.dot(q.T).T + t

  return q1



In [6]:
def show_pointcloud(p,q):
  color1=np.tile(np.array([[1, 0.5, 0]]), (len(p), 1))
  po = o3d.geometry.PointCloud()
  po.points = o3d.utility.Vector3dVector(p)
  po.colors =o3d.utility.Vector3dVector(color1)
  points1=np.asarray(po.points)
  color1= np.asarray(po.colors)

  color2=np.tile(np.array([[0.5, 0, 1]]), (len(q), 1))
  qo = o3d.geometry.PointCloud()
  qo.points = o3d.utility.Vector3dVector(q)
  qo.colors =o3d.utility.Vector3dVector(color2)
  points2 = np.asarray(qo.points)
  color2=np.asarray(qo.colors)

  points=np.concatenate((points1,points2))
  colors=np.concatenate((color1,color2))
  fig = go.Figure(
    data=[
      go.Scatter3d(
        x=points[:,0], y=points[:,1], z=points[:,2],
        mode='markers',
        marker=dict(size=1,color=colors)
  )
  ],
    layout=dict(
      scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
  )
  )
  )
  fig.show()


In [7]:
def ICP(p, q, iterations=10):
    def matches(tr, q):
        dd, ii = tr.query(q, k=1, workers=-1)
        return [(ii[i], i) for i in range(ii.shape[0])]

    nq = q.shape[0]

    qs = np.empty((iterations, nq,3))
    pm = p.mean(axis=0)
    tr = kdtree(p - pm)
    qj = q.copy()
    for j in range(iterations):


        qm = qj.mean(axis=0)
        cc = cov(p - pm, qj - qm, matches(tr, qj - qm))
        qj=transform(cc,pm,qj,qm)

        qs[j]=qj
        mse=np.mean((p - qj)**2)
        print('The iter {} and mean_square_error is {}'.format(j,mse))


    return qs

iterations=15
qs = ICP(p, q, iterations = iterations)

The iter 0 and mean_square_error is 0.5987887879456639
The iter 1 and mean_square_error is 0.2968920733295761
The iter 2 and mean_square_error is 0.15832409176807694
The iter 3 and mean_square_error is 0.08406474834091093
The iter 4 and mean_square_error is 0.05182694789982707
The iter 5 and mean_square_error is 0.03566731783300281
The iter 6 and mean_square_error is 0.019857575069301
The iter 7 and mean_square_error is 0.012490921292639479
The iter 8 and mean_square_error is 0.008025578940278244
The iter 9 and mean_square_error is 0.004940545387369426
The iter 10 and mean_square_error is 0.0026494044821377474
The iter 11 and mean_square_error is 0.000862375441018028
The iter 12 and mean_square_error is 4.941991789235947e-05
The iter 13 and mean_square_error is 8.766615693632698e-09
The iter 14 and mean_square_error is 2.142546139693087e-30


In [8]:
show_pointcloud(p,q)
for i in range(iterations):

  show_pointcloud(p,qs[i])